In [3]:
import requests
import csv
from datetime import datetime, timedelta
import time
from requests.exceptions import ConnectionError, Timeout

# List of 10 major cities in Nepal with their coordinates
cities = [
    {"name": "Kathmandu", "lat": 27.7172, "lon": 85.3240},
    {"name": "Pokhara", "lat": 28.2096, "lon": 83.9856},
    {"name": "Lalitpur", "lat": 27.6588, "lon": 85.3247},
    {"name": "Bharatpur", "lat": 27.6833, "lon": 84.4333},
    {"name": "Biratnagar", "lat": 26.4525, "lon": 87.2714},
    {"name": "Birgunj", "lat": 27.0449, "lon": 84.8672},
    {"name": "Janakpur", "lat": 26.7122, "lon": 85.9248},
    {"name": "Ghorahi", "lat": 28.0333, "lon": 82.4833},
    {"name": "Hetauda", "lat": 27.4368, "lon": 85.0026},
    {"name": "Dhangadhi", "lat": 28.7016, "lon": 80.5936}
]

# Date range for 2023
start_date = "2023-01-01"
end_date = "2023-12-31"

# Weather code mapping to weather types
weather_code_map = {
    0: "Clear",
    1: "Cloudy",
    2: "Cloudy",
    3: "Cloudy",
    45: "Fog",
    48: "Fog",
    51: "Drizzle",
    53: "Drizzle",
    55: "Drizzle",
    56: "Drizzle",
    57: "Drizzle",
    61: "Rain",
    63: "Rain",
    65: "Rain",
    66: "Rain",
    67: "Rain",
    71: "Snow",
    73: "Snow",
    75: "Snow",
    77: "Snow",
    80: "Rain Showers",
    81: "Rain Showers",
    82: "Rain Showers",
    85: "Snow Showers",
    86: "Snow Showers",
    95: "Thunderstorm",
    96: "Thunderstorm",
    99: "Thunderstorm"
}

# Function to map weather code to weather type
def map_weather_code(code):
    return weather_code_map.get(code, "Unknown")

# List to hold all data rows
data_rows = []

# API endpoint
api_url = "https://archive-api.open-meteo.com/v1/archive"

# Function to fetch data with retries
def fetch_data_with_retries(city, retries=3, delay=5):
    for attempt in range(retries):
        try:
            params = {
                "latitude": city["lat"],
                "longitude": city["lon"],
                "start_date": start_date,
                "end_date": end_date,
                "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,weathercode",
                "hourly": "relativehumidity_2m",
                "timezone": "auto"
            }
            response = requests.get(api_url, params=params, timeout=10)
            response.raise_for_status()  # Raise exception for HTTP errors
            return response.json()
        except (ConnectionError, Timeout) as e:
            print(f"Connection error for {city['name']}: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed for {city['name']}: {e}")
            break
    return None

# Loop through each city
for city in cities:
    print(f"Processing city: {city['name']}")
    data = fetch_data_with_retries(city)
    if data is None:
        print(f"Failed to fetch data for {city['name']} after retries.")
        continue
    
    # Extract daily and hourly data
    daily_data = data["daily"]
    hourly_data = data["hourly"]
    
    # Loop through each day
    for i, date in enumerate(daily_data["time"]):
        # Daily data
        temp_max = daily_data["temperature_2m_max"][i]
        temp_min = daily_data["temperature_2m_min"][i]
        temp_mean = (temp_max + temp_min) / 2 if temp_max is not None and temp_min is not None else None
        rainfall = daily_data["precipitation_sum"][i]
        weather_code = daily_data["weathercode"][i]
        weather_type = map_weather_code(weather_code)
        
        # Get hourly data for this day
        start_time = datetime.strptime(date, "%Y-%m-%d")
        end_time = start_time + timedelta(days=1)
        hourly_times = [datetime.strptime(t, "%Y-%m-%dT%H:%M") for t in hourly_data["time"]]
        indices = [j for j, t in enumerate(hourly_times) if start_time <= t < end_time]
        humidity_values = [hourly_data["relativehumidity_2m"][j] for j in indices if hourly_data["relativehumidity_2m"][j] is not None]
        humidity_mean = sum(humidity_values) / len(humidity_values) if humidity_values else None
        
        # Append row to data_rows
        row = [
            date,
            city["name"],
            round(rainfall, 1) if rainfall is not None else None,
            round(temp_mean, 1) if temp_mean is not None else None,
            round(humidity_mean) if humidity_mean is not None else None,
            weather_type
        ]
        data_rows.append(row)
    
    # Add a delay between cities to avoid rate limiting
    time.sleep(5)

# Write data to CSV file
csv_filename = "nepal_weather_2023.csv"
headers = ["Date", "City", "Rainfall_mm", "Temperature_C", "Humidity_%", "Weather_Type"]

with open(csv_filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(headers)
    writer.writerows(data_rows)

print(f"CSV file '{csv_filename}' has been generated successfully.")

Processing city: Kathmandu
Processing city: Pokhara
Processing city: Lalitpur
Processing city: Bharatpur
Processing city: Biratnagar
Connection error for Biratnagar: HTTPSConnectionPool(host='archive-api.open-meteo.com', port=443): Read timed out. (read timeout=10). Retrying in 5 seconds...
Processing city: Birgunj
Processing city: Janakpur
Processing city: Ghorahi
Processing city: Hetauda
Processing city: Dhangadhi
Connection error for Dhangadhi: HTTPSConnectionPool(host='archive-api.open-meteo.com', port=443): Read timed out. (read timeout=10). Retrying in 5 seconds...
CSV file 'nepal_weather_2023.csv' has been generated successfully.
